#### Rewrite SQL stored procedure to pyspark

PRC_SYNC_DATA & PRC_SEL_XML_PARSER_RESULT

In [0]:
PROJECT_NAME = 'BIFC_OSCE'
DB = 'BIF_CONNECT' 
START_DATE = '2022-01-20' 
END_DATE = '2022-01-22'

In [0]:
start_date= (datetime.now() + timedelta(days = -30)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")
print("start_date:",start_date)
print("end_date:",end_date)

start_date: 2022-03-19
end_date: 2022-04-18

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, StringType, ArrayType, LongType, TimestampType, BinaryType, IntegerType, DateType
from datetime import datetime, timedelta
# from pyspark.sql import Window
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

from functools import reduce
from pyspark.sql import DataFrame

from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id, row_number

In [0]:
#synapse 
spark.conf.set('fs.azure.account.key.gdwpocdls.dfs.core.windows.net', dbutils.secrets.get(scope = 'BIFC-asagdwpocws-etl', key = 'StorageAccount-gdwpocdls-AccessKey'));
_url = dbutils.secrets.get(scope = 'BIFC-asagdwpocws-etl', key = 'Synapse-asagdwpocetl-ConnectionString');


**Load Synaspe Table**

In [0]:
df_raw = (spark.read
          .format('com.databricks.spark.sqldw')
          .option('url', _url)
          .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
          .option('useAzureMSI', 'true')
          .option('dbTable', 'BIFC.TBL_DATA_RAW')
          .load()
          .filter(F.col('file_date') > START_DATE)
          .filter(F.col('file_date') < END_DATE)
          )

df_raw.createOrReplaceTempView('TBL_DATA_RAW')
# display(df_raw)

df_group = (spark.read
            .format('com.databricks.spark.sqldw')
            .option('url', _url)
            .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
            .option('useAzureMSI', 'true')
            .option('dbTable', 'BIFC_ETL.TBL_PROJECT_GROUP')
            .load()
            .filter(F.col('Project_Name') == PROJECT_NAME)
            )
display(df_group)
df_group.createOrReplaceTempView('TBL_PROJECT_GROUP')

df_parse_config = (spark.read
            .format('com.databricks.spark.sqldw')
                    .option('url', _url)
                    .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
                    .option('useAzureMSI', 'true')
                    .option('dbTable', 'BIFC_ETL.TBL_XML_PARSER_CONFIG')
                    .load()
                    .withColumn("sort",F.col("sort").cast(IntegerType()))
                   )
display(df_parse_config)

df_raw_config = (spark.read
                    .format('com.databricks.spark.sqldw')
                    .option('url', _url)
                    .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
                    .option('useAzureMSI', 'true')
                    .option('dbTable', 'BIFC_ETL.TBL_XML_RAW_CONFIG')
                    .load()
                   )
display(df_raw_config)

Project_Group_id,Project_Name,Group_Name,Name_Value_Pair_Table_Name,Flat_Table_Name,Data_Source_View,With_File_Date,Table_Schema
134,BIFC_OSCE,OSCE_2.0_Endpoint_OS,null,OSCE_Endpoint_OS,null,true,OSCE
161,BIFC_OSCE,OSCE_2.0_WR_Internal,OSCE_WR_Internal,null,null,true,OSCE
240,BIFC_OSCE,OSCE_1.0_Feedback_Main,null,OSCE_Feedback_Main,null,true,OSCE
272,BIFC_OSCE,OSCE_2.0_Server_Edge_Agent,OSCE_Server_Edge_Agent,null,null,true,OSCE
432,BIFC_OSCE,OSCE_2.0_iES_Endpoint_Policy_DB_Size,null,OSCE_iES_Endpoint_Policy_DB_Size,null,true,OSCE
440,BIFC_OSCE,OSCE_2.0_iES_Server_Impact_Eval_Source,null,OSCE_iES_Server_Impact_Eval_Source,null,true,OSCE
138,BIFC_OSCE,OSCE_2.0_Feedback_Main,null,OSCE_Feedback_Main,null,true,OSCE
165,BIFC_OSCE,OSCE_2.0_Server_OPP,OSCE_Server_OPP,null,null,true,OSCE
428,BIFC_OSCE,OSCE_2.0_iAC_Server_TMCSS_Usage,null,OSCE_iAC_Server_TMCSS_Usage,null,true,OSCE
436,BIFC_OSCE,OSCE_2.0_iES_Server_ADE_Detect,null,OSCE_iES_Server_ADE_Detect,null,true,OSCE


project_group_id,xpath_id,column_name,data_type,data_length,sort,boolean_string,category
436,3967,Category_ID,varchar,44,1,null,null
425,3932,AC,varchar,65,1,null,null
439,3978,Criteria_Used,int,null,4,null,null
138,3924,iAC_Agent_Enabled_Count,int,null,108,null,null
138,4004,S_Update_Source_Type,int,null,136,null,null
435,3962,Count,int,null,1,null,null
138,3931,iAC_Config_Rule_Enabled,varchar,1,115,true,null
138,4072,iAC_Config_Rule_Enabled,varchar,1,141,true,null
443,4080,Rule_ID,int,null,2,null,null
429,3966,AC,varchar,65,1,null,null


xpath_id,source,xml_version,level,xpath,raw_group_root,group_root,master_detail,default_column_name,default_data_type,default_data_length,sample_value,group_level
3,TMEAC,1.0,3,/pr/product/AC,/pr/product,/pr,M,product_AC,varchar,84,EI-27JK-N7SSH-3RW2D-MZ9VL-FQG35-ZNFNF,2
11,TMEAC,1.0,5,/pr/product/policies/policy/@log,/pr/product/policies/policy,/pr/product/policies,D,policy_log,varchar,89,SpecificRulesOnly,4
19,TMEAC,1.0,5,/pr/product/rules/rule/@size,/pr/product/rules/rule,/pr/product/rules,D,rule_size,int,null,152356,4
27,TMEAC,1.0,6,/pr/product/targets/group/distribution/@name,/pr/product/targets/group/distribution,/pr/product/targets,D,distribution_name,varchar,39,version,5
35,DS,1.0,6,/Data/Products/Product/EndPoint/ActivatedSeat/@Agents,/Data/Products/Product/EndPoint/ActivatedSeat,/Data/Products/Product/EndPoint/ActivatedSeat,M,ActivatedSeat_Agents,int,null,88154,5
43,DS,1.0,7,/Data/Products/Product/EndPoint/ModuleUsage/Item/@EnabledCount,/Data/Products/Product/EndPoint/ModuleUsage/Item,/Data/Products/Product/EndPoint/ModuleUsage/Item,D,Item_EnabledCount,int,null,12162,6
51,DS,1.0,5,/Data/Products/Product/EndPoint/TenantName,/Data/Products/Product/EndPoint,/Data/Products/Product/EndPoint,M,EndPoint_TenantName,nvarchar,95,Cached Tenant - C7BF42A8-AB0C-C0B5-0B3F-FB0AEADA4407,4
59,DS,1.0,5,/Data/Products/Product/Server/LastInstalled,/Data/Products/Product/Server,/Data/Products/Product/Server,M,Server_LastInstalled,datetime,null,2014-12-10T08:30:37.580-05:00,4
67,DS,1.0,7,/Data/Products/Product/Server/PLS/Module/@Version,/Data/Products/Product/Server/PLS/Module,/Data/Products/Product/Server/PLS/Module,D,Module_Version,varchar,62,1.5.272,6
75,TMCM,1.0,6,/pr/products/product/ACs/AC/@Key,/pr/products/product/ACs/AC,/pr/products/product/ACs/AC,D,AC_Key,varchar,56,TM-GR6S-945CR-4FFXD-VT3VB-H55LU-QU9VF,5


**Generate Result Type 1 & Result Type 2 table create script**

In [0]:
#ResultType 1:Name/Value Pair
df1 = (df_group
    .filter('Name_Value_Pair_Table_Name is not null')
    .withColumn('ResultType', F.lit('1'))
    .withColumn('Table_Name', F.col('Name_Value_Pair_Table_Name'))
    .withColumn('With_File_Date', F.when(F.col('With_File_Date').isNull(), F.lit(False)).otherwise(F.col('With_File_Date'))) 
    .select('Group_Name','Table_Schema','Table_Name','ResultType','Name_Value_Pair_Table_Name','Data_Source_View','With_File_Date','project_group_id')
     )

display(df1.limit(5))

Group_Name,Table_Schema,Table_Name,ResultType,Name_Value_Pair_Table_Name,Data_Source_View,With_File_Date,project_group_id
OSCE_2.0_WR_External,OSCE,OSCE_WR_External,1,OSCE_WR_External,null,true,160
OSCE_2.0_Server_Integrated_Server,OSCE,OSCE_Server_Integrated_Server,1,OSCE_Server_Integrated_Server,null,true,164
OSCE_2.0_Server_Agent_Global_Setting,OSCE,OSCE_Server_Agent_Global_Setting,1,OSCE_Server_Agent_Global_Setting,null,true,163
OSCE_2.0_Server_OPP,OSCE,OSCE_Server_OPP,1,OSCE_Server_OPP,null,true,165
OSCE_2.0_Endpoint_BM,OSCE,OSCE_Endpoint_BM,1,OSCE_Endpoint_BM,null,true,158


In [0]:
df1_rdd = df1.collect()

#ResultType 1:Name/Value Pair

type1_create_table_list = []

for row in df1_rdd:

    data={}
    Group_Name = row.Group_Name
    Table_Name = row.Table_Name
    Table_Schema = row.Table_Schema
    Data_Source_View = row.Data_Source_View
    create = ''
    sql = '[File_ID] [bigint]'
    col = '[File_ID]'
    fullTableName = f'{DB}.{row.Table_Name}'
    project_group_id = row.project_group_id
    if row.With_File_Date:
        sql = sql + ',[File_Date] [date]'
        col = col + ',[File_Date]'
    
    print(fullTableName)

    if ( Data_Source_View == None ):

        df_config_table = (df_parse_config.alias('m')
                          .filter(f'project_group_id=={project_group_id}')
                          )

        df_config_table_rdd = df_config_table.select('category','data_type','data_length').collect()

        category = df_config_table_rdd[1][0]
        data_type = df_config_table_rdd[1][1]
        data_length = df_config_table_rdd[1][2]

        if (len(category)) > 0:
            sql = sql + ', [group] [varchar](100) NOT NULL,[row_id] [uniqueidentifier] NOT NULL, [column_name] [varchar](1000) NULL, [column_value] [nvarchar](4000) NULL'
        else:
            colcount = '[count] [bigint] NULL' if data_type == 'int' else f'[Value] [nvarchar](20) NULL'
            sql = sql + ', [Category] [varchar](300) NOT NULL,[Name] [varchar](1000) NULL,' + colcount
 

    create = f'create table {fullTableName}({sql})WITH(DISTRIBUTION = ROUND_ROBIN,CLUSTERED COLUMNSTORE INDEX)'
    data['fullTableName'] = fullTableName
    data['project_group_id'] = project_group_id
    data['group_name'] = Group_Name
    data['result_type'] = 1
    data['col'] = ''
    data['create'] = create
    data['data_source_view'] = Data_Source_View
    data['with_file_date'] = row.With_File_Date    
    type1_create_table_list.append(data)


BIF_CONNECT.OSCE_Endpoint_BM
BIF_CONNECT.OSCE_Server_Smart_Protection
BIF_CONNECT.OSCE_Server_Agent_Global_Setting
BIF_CONNECT.OSCE_Server_Integrated_Server
BIF_CONNECT.OSCE_WR_Internal
BIF_CONNECT.OSCE_Server_Edge_Agent
BIF_CONNECT.OSCE_Endpoint_Setting
BIF_CONNECT.OSCE_Server_BT_SO
BIF_CONNECT.OSCE_WR_External
BIF_CONNECT.OSCE_Server_OPP

In [0]:
#ResultType 2:Flat table

df2 = (df_group
    .filter('Flat_Table_Name is not null')
    .withColumn('ResultType', F.lit('2'))
    .withColumn('Table_Name', F.col('Flat_Table_Name'))
    .withColumn('With_File_Date', F.when(F.col('With_File_Date').isNull(), F.lit(False)).otherwise(F.col('With_File_Date'))) 
    .select('Group_Name','Table_Schema','Table_Name','ResultType','Name_Value_Pair_Table_Name','Data_Source_View','With_File_Date','project_group_id')
     )
display(df2.limit(5))

Group_Name,Table_Schema,Table_Name,ResultType,Name_Value_Pair_Table_Name,Data_Source_View,With_File_Date,project_group_id
OSCE_2.0_Feedback_Main,OSCE,OSCE_Feedback_Main,2,null,null,true,138
OSCE_2.0_iAC_Server_TMCSS_Usage,OSCE,OSCE_iAC_Server_TMCSS_Usage,2,null,null,true,428
OSCE_2.0_iES_Server_ADE_Detect,OSCE,OSCE_iES_Server_ADE_Detect,2,null,null,true,436
OSCE_2.0_iES_Endpoint_Latest_Policy,OSCE,OSCE_iES_Endpoint_Latest_Policy,2,null,null,true,470
OSCE_2.0_Server_Hotfix,OSCE,OSCE_Server_Hotfix,2,null,null,true,139


In [0]:
windowSpec  = Window.partitionBy("m.column_name").orderBy(F.col("data_type").desc())

df2_rdd = df2.collect()

# ResultType 2:Flat table ceate table schema

type2_create_table_list = []

for row in df2_rdd:

    data={}
    Group_Name = row.Group_Name
    Table_Name = row.Table_Name
    Table_Schema = row.Table_Schema
    Data_Source_View = row.Data_Source_View
    create = ''
    sql = '[File_ID] [bigint]'
    col = '[File_ID]'
    fullTableName = f'{DB}.{row.Table_Name}'
    project_group_id = row.project_group_id
   
    if row.With_File_Date:
        sql = sql + ',[File_Date] [date]'
        col = col + ',[File_Date]'
    
    print(fullTableName)

    if ( Data_Source_View == None ):

        df_config_sort = df_parse_config.groupBy('project_group_id','column_name').agg(F.min('sort').alias('sort'))
        
        df_config_table = (df_parse_config.alias('m')
                          .filter(f'project_group_id=={project_group_id}')
                          .join(df_config_sort.alias('s'), [F.col('m.project_group_id')==F.col('s.project_group_id'),
                                                                        F.col('m.column_name')==F.col('s.column_name')], 'inner')
                          .select('m.column_name','data_type','data_length','s.sort')
                          .withColumn("data_length",F.col("data_length").cast(IntegerType()))
                          .withColumn("id",row_number().over(windowSpec))
                          .filter('id == 1')
                          .orderBy(F.col("sort").asc())
                          )

        df_config_table_rdd = df_config_table.collect()
        for t in df_config_table_rdd:
            #print(t)
            lenght = f'({t.data_length})' if t.data_length != None else ''
            sql = sql + f',[{t.column_name}] [{t.data_type}]{lenght}' 
            col = col + f',[{t.column_name}]'
        create = f'create table {fullTableName}({sql})WITH(DISTRIBUTION = ROUND_ROBIN,CLUSTERED COLUMNSTORE INDEX)'
    
#     else:
#         query = f"""select COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH 
#             from INFORMATION_SCHEMA.COLUMNS
#             where TABLE_SCHEMA = {DB} and TABLE_NAME = {Table_Name}
#             -- QUOTENAME(TABLE_SCHEMA) + '.' + QUOTENAME(TABLE_NAME) = @data_source_view 
#             and [COLUMN_NAME] not in ('file_id', 'file_date')"""
        
#         df_config_table = (spark.read
#                 .format('com.databricks.spark.sqldw')
#                 .option('url', _url)
#                 .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
#                 .option('useAzureMSI', 'true')
#                 .option('query', query)
#                 .load()
#                );
        
#         df_config_table_rdd = df_config_table.collect()
#         for t in df_config_table_rdd:
#         #     print(t)
#             lenght = f'({t.CHARACTER_MAXIMUM_LENGTH})' if t.CHARACTER_MAXIMUM_LENGTH != None else ''
#             sql = sql + f',[{t.column_name}] [{t.data_type}]{lenght}' 

    data['fullTableName'] = fullTableName
    data['project_group_id'] = project_group_id
    data['group_name'] = Group_Name
    data['result_type'] = 2
    data['col'] = col
    data['create'] = create
    data['data_source_view'] = Data_Source_View
    data['with_file_date'] = row.With_File_Date
    type2_create_table_list.append(data)
    

BIF_CONNECT.OSCE_Endpoint_Version
BIF_CONNECT.OSCE_iAC_AC
BIF_CONNECT.OSCE_iES_Endpoint_Policy_Event_Recording
BIF_CONNECT.OSCE_iES_Server_Investigation
BIF_CONNECT.OSCE_Endpoint_Config
BIF_CONNECT.OSCE_iAC_Agent_Rule_Method_Result
BIF_CONNECT.OSCE_iES_Endpoint_Policy_Meta_Upload
BIF_CONNECT.OSCE_iVP_AC
BIF_CONNECT.OSCE_Endpoint_Plugin
BIF_CONNECT.OSCE_iAC_Server_Rule_Method_Usage
BIF_CONNECT.OSCE_iES_Endpoint_Policy_Meta_Upload_Frequency
BIF_CONNECT.OSCE_iVP_Server_IPS_Rule
BIF_CONNECT.OSCE_AC
BIF_CONNECT.OSCE_Client_OS_10
BIF_CONNECT.OSCE_iES_Endpoint_Policy_ADE
BIF_CONNECT.OSCE_iES_Server_Impact_Eval_Evaluate
BIF_CONNECT.OSCE_iES_Server_Impact_Eval_Operator
BIF_CONNECT.OSCE_Endpoint_OS
BIF_CONNECT.OSCE_Feedback_Main
BIF_CONNECT.OSCE_iES_Endpoint_Policy_DB_Size
BIF_CONNECT.OSCE_iES_Server_Impact_Eval_Source
BIF_CONNECT.OSCE_Server_Plugin
BIF_CONNECT.OSCE_AC
BIF_CONNECT.OSCE_iES_Endpoint_Agent
BIF_CONNECT.OSCE_iES_Server_Impact_Eval_Criteria
BIF_CONNECT.OSCE_iES_Server_Impact_Eval_Criteria_Type
BIF_CONNECT.OSCE_Server_Hotfix
BIF_CONNECT.OSCE_iES_AC
BIF_CONNECT.OSCE_iES_Server_API
BIF_CONNECT.OSCE_iES_Server_Impact_Eval_Access_Scope
BIF_CONNECT.OSCE_Feedback_Main
BIF_CONNECT.OSCE_iAC_Server_TMCSS_Usage
BIF_CONNECT.OSCE_iES_Server_ADE_Detect
BIF_CONNECT.OSCE_iES_Endpoint_Latest_Policy

In [0]:
create_list = type1_create_table_list + type2_create_table_list
create_rdd= spark.sparkContext.parallelize(create_list)
create_df = (spark
             .createDataFrame(create_rdd)
            )
display(create_df.filter("result_type == 2"))

col,create,data_source_view,fullTableName,group_name,project_group_id,result_type,with_file_date
"[File_ID],[File_Date],[Client_Version_Count],[Client_Version_Build]","create table BIF_CONNECT.OSCE_Endpoint_Version([File_ID] [bigint],[File_Date] [date],[Client_Version_Count] [int],[Client_Version_Build] [varchar](17))WITH(DISTRIBUTION = ROUND_ROBIN,CLUSTERED COLUMNSTORE INDEX)",null,BIF_CONNECT.OSCE_Endpoint_Version,OSCE_2.0_Endpoint_Version,135,2,true
"[File_ID],[File_Date],[AC]","create table BIF_CONNECT.OSCE_iAC_AC([File_ID] [bigint],[File_Date] [date],[AC] [varchar](65))WITH(DISTRIBUTION = ROUND_ROBIN,CLUSTERED COLUMNSTORE INDEX)",null,BIF_CONNECT.OSCE_iAC_AC,OSCE_2.0_iAC_AC,425,2,true
"[File_ID],[File_Date],[Count],[Recording_Enabled],[Platform_Type]","create table BIF_CONNECT.OSCE_iES_Endpoint_Policy_Event_Recording([File_ID] [bigint],[File_Date] [date],[Count] [int],[Recording_Enabled] [varchar](1),[Platform_Type] [varchar](13))WITH(DISTRIBUTION = ROUND_ROBIN,CLUSTERED COLUMNSTORE INDEX)",null,BIF_CONNECT.OSCE_iES_Endpoint_Policy_Event_Recording,OSCE_2.0_iES_Endpoint_Policy_Event_Recording,433,2,true
"[File_ID],[File_Date],[Count],[Is_Scheduled_Task],[Scan_Type]","create table BIF_CONNECT.OSCE_iES_Server_Investigation([File_ID] [bigint],[File_Date] [date],[Count] [int],[Is_Scheduled_Task] [varchar](1),[Scan_Type] [varchar](28))WITH(DISTRIBUTION = ROUND_ROBIN,CLUSTERED COLUMNSTORE INDEX)",null,BIF_CONNECT.OSCE_iES_Server_Investigation,OSCE_2.0_iES_Server_Investigation,441,2,true
"[File_ID],[File_Date]",,[BIFC_ETL].[vw_OSCE_Endpoint_Config],BIF_CONNECT.OSCE_Endpoint_Config,OSCE_2.0_Endpoint_Config,136,2,true
"[File_ID],[File_Date],[Count],[Match_Method_ID]","create table BIF_CONNECT.OSCE_iAC_Agent_Rule_Method_Result([File_ID] [bigint],[File_Date] [date],[Count] [int],[Match_Method_ID] [int])WITH(DISTRIBUTION = ROUND_ROBIN,CLUSTERED COLUMNSTORE INDEX)",null,BIF_CONNECT.OSCE_iAC_Agent_Rule_Method_Result,OSCE_2.0_iAC_Agent_Rule_Method_Result,426,2,true
"[File_ID],[File_Date],[Count],[Upload_Enabled],[Platform_Type]","create table BIF_CONNECT.OSCE_iES_Endpoint_Policy_Meta_Upload([File_ID] [bigint],[File_Date] [date],[Count] [int],[Upload_Enabled] [int],[Platform_Type] [varchar](13))WITH(DISTRIBUTION = ROUND_ROBIN,CLUSTERED COLUMNSTORE INDEX)",null,BIF_CONNECT.OSCE_iES_Endpoint_Policy_Meta_Upload,OSCE_2.0_iES_Endpoint_Policy_Meta_Upload,434,2,true
"[File_ID],[File_Date],[AC]","create table BIF_CONNECT.OSCE_iVP_AC([File_ID] [bigint],[File_Date] [date],[AC] [varchar](65))WITH(DISTRIBUTION = ROUND_ROBIN,CLUSTERED COLUMNSTORE INDEX)",null,BIF_CONNECT.OSCE_iVP_AC,OSCE_2.0_iVP_AC,442,2,true
"[File_ID],[File_Date],[Plugin_Count],[Plugin_Name],[Plugin_Status]","create table BIF_CONNECT.OSCE_Endpoint_Plugin([File_ID] [bigint],[File_Date] [date],[Plugin_Count] [int],[Plugin_Name] [varchar](21),[Plugin_Status] [int])WITH(DISTRIBUTION = ROUND_ROBIN,CLUSTERED COLUMNSTORE INDEX)",null,BIF_CONNECT.OSCE_Endpoint_Plugin,OSCE_2.0_Endpoint_Plugin,137,2,true
"[File_ID],[File_Date],[Base_Action],[Rule_Method]","create table BIF_CONNECT.OSCE_iAC_Server_Rule_Method_Usage([File_ID] [bigint],[File_Date] [date],[Base_Action] [int],[Rule_Method] [int])WITH(DISTRIBUTION = ROUND_ROBIN,CLUSTERED COLUMNSTORE INDEX)",null,BIF_CONNECT.OSCE_iAC_Server_Rule_Method_Usage,OSCE_2.0_iAC_Server_Rule_Method_Usage,427,2,true


**Create table in synapse if doesn't exist**

Create table script is DDL, Spark driver is not a general-purpose database library, we cannot use spark.write or spark.read to run DDL or execute Stored Procedure. But the Spark driver is built on top of the JDBC driver, which you can use directly in either Scala or Java

In [0]:
%scala
import java.sql.DriverManager
import java.sql.Connection
import java.util.Properties
 
 Create the JDBC URL without passing in the user and password parameters.
val jdbcUrl = dbutils.secrets.get(scope = "BIFC-asagdwpocws-etl", key = "Synapse-asagdwpocetl-ConnectionString");

 // Create a Properties() object to hold the parameters.
val connectionProperties = new Properties()
val driverClass = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

connectionProperties.setProperty("Driver", driverClass)
     
val con = DriverManager.getConnection(jdbcUrl, connectionProperties)
val stmt = con.createStatement()

for ( var row <- create_df.collect()){
    
    val fullTableName = row.fullTableName
    val data_source_view = row.data_source_view
    val project_group_id = row.project_group_id
    val create = row.create
    val result_type = row.result_type
    
    val query = f"""select OBJECT_ID(N'%fullTableName%s') as ObjectId """
  
   
    val df_table_exist = (spark.read
                    .format("com.databricks.spark.sqldw")
                    .option("url", _url)
                    .option("tempDir", "abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs")
                    .option("useAzureMSI", "true")
                    .option("query", query)
                    .load()
                   );
        
    val df_table_exist_rdd = df_table_exist.collect()
    val is_table_exist: Boolean = true
    if (df_table_exist_rdd[0][0] != None){ 
        is_table_exist = true 
    }else{
       is_table_exist = false
    }
                      
    if (is_table_exist == false){
        println(f"%fullTableName%s %is_table_exist%s")
        stmt.execute(create)
    }
}
    

In [0]:
%scala
//  import java.sql.DriverManager
//  import java.sql.Connection
//  import java.util.Properties
 
//  // Create the JDBC URL without passing in the user and password parameters.
//  val jdbcUrl = dbutils.secrets.get(scope = "BIFC-asagdwpocws-etl", key = "Synapse-asagdwpocetl-ConnectionString");

//  // Create a Properties() object to hold the parameters.
//  val connectionProperties = new Properties()

// val driverClass = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
//  connectionProperties.setProperty("Driver", driverClass)
     
//  val con = DriverManager.getConnection(jdbcUrl, connectionProperties)
//  val stmt = con.createStatement()
     
//  stmt.execute("create table BIF_CONNECT.OSCE_Endpoint_Setting5 ([File_ID] [bigint],[File_Date] [date], [group] [varchar](100) NOT NULL,[column_name] [varchar](1000) NULL, [column_value] [nvarchar](1000) NULL)WITH(DISTRIBUTION = ROUND_ROBIN,CLUSTERED COLUMNSTORE INDEX)")

**Insert data into the table in synapse**

use function to get the parse result:  
df = parse_data(parse_data(project_name, group_name, project_group_id, result_type, startdate, enddata, with_file_date)

In [0]:
# PRC_SEL_XML_PARSER_RESULT

create_df_1 = create_df.filter("result_type==1") 
create_df_2 = create_df.filter("result_type==2") 

display(create_df_2)

In [0]:

for row in create_df_2.collect():
    
   
    fullTableName = row.fullTableName
    data_source_view = row.data_source_view
    project_group_id = row.project_group_id
    col =row.col
    result_type = row.result_type
    with_file_date = row.with_file_date
    
    print(project_group_id, fullTableName)
    
    if data_source_view == None:
        # parse _data det dataframe from xml source data
        try:
            parse_df = parse_data(None,None,project_group_id,result_type,None,None,with_file_date)
            display(parse_df)
            if (result_type ==2):
                sql = f'insert into {fullTableName} ({col})'
            else: # result_type ==1 
                sql = f'insert into {fullTableName}'
            print(sql)
            (df1.write
                .format("com.databricks.spark.sqldw")
                .option("useAzureMSI", "true")
                .option("dbTable", fullTableName)
                .option("url", _url)
                .option("tempdir", "abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs")
                .save()
            )
        except Exception as e:
            print(e)
    else:
        print(f'{fullTableName} {data_source_view}' )

**Parse XML result def**

In [0]:
df = parse_data(None,None,159,1,START_DATE,END_DATE,True)
display(df)

In [0]:
df = parse_data(None,None,159,2,START_DATE,END_DATE,True)
display(df)

In [0]:
from functools import reduce
from pyspark.sql import DataFrame

from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id, row_number
    
def parse_data(project_name, group_name, project_group_id, result_type, startdate, enddata, with_file_date):

#     project_name = ''
#     group_name = ''
#     project_group_id = 140
#     result_type = 2
#     with_file_date = True

  
    if project_group_id == None:
        df_group = (spark.read
            .format('com.databricks.spark.sqldw')
            .option('url', _url)
            .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
            .option('useAzureMSI', 'true')
            .option('dbTable', 'BIFC_ETL.TBL_PROJECT_GROUP')
            .load()
         .filter(F.col('Project_Name') == project_name)
         .filter(F.col('Group_Name') == group_name)
         .select('Project_Group_id')
        )
        project_group_id = df_group.collect()[0][0]

    df_parse_config = (spark.read
            .format('com.databricks.spark.sqldw')
                    .option('url', _url)
                    .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
                    .option('useAzureMSI', 'true')
                    .option('dbTable', 'BIFC_ETL.TBL_XML_PARSER_CONFIG')
                    .load()
                    .withColumn("sort",F.col("sort").cast(IntegerType()))
                    .filter(f'project_group_id=={project_group_id}')
                   )

    df_raw_config = (spark.read
                        .format('com.databricks.spark.sqldw')
                        .option('url', _url)
                        .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
                        .option('useAzureMSI', 'true')
                        .option('dbTable', 'BIFC_ETL.TBL_XML_RAW_CONFIG')
                        .load()
                       )


    df_config_table = (df_parse_config.alias('m')
                      .filter(f'project_group_id=={project_group_id}')
                      .join(df_raw_config.alias('r'), [F.col('m.xpath_id')==F.col('r.xpath_id')], 'inner')
                      .drop(df_raw_config.xpath_id)
                      )

    df_config_table_list = df_config_table.select('source','xml_version','category').collect()

    source = df_config_table_list[0][0]
    xml_version =  df_config_table_list[0][1]
    category =  df_config_table_list[0][2]

    # df_config_table_list = df_config_table.filter("master_detail = 'D'").orderBy(F.col("level").desc()).select('raw_group_root','level').collect()
    # root_node = df_config_table_list[0][0]
    # level =  df_config_table_list[0][1]
    df_raw = (spark.read
          .format('com.databricks.spark.sqldw')
          .option('url', _url)
          .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
          .option('useAzureMSI', 'true')
          .option('dbTable', 'BIFC.TBL_DATA_RAW')
          .load()
          .filter(F.col('source') == source)
          .filter(F.col('xml_version') == xml_version )
          .filter(F.col('file_date') > startdate)
          .filter(F.col('file_date') < enddata)
          .withColumn('file_date', F.date_format(F.col('file_date'),"yyyy/MM/dd"))
          )

    df_raw.createOrReplaceTempView('TBL_DATA_RAW')
   
    
    if category != None and result_type == 2:
        df_t_col = (df_config_table
                    .withColumn('category', F.lit(''))
                    .withColumn('column_name', F.when(F.col('category').isNotNull(), F.concat(F.col('category'),F.lit('_'),F.col('column_name'))).otherwise(F.col('column_name')))
                    .groupBy('category','column_name','xpath_id').agg(F.min('sort').alias('sort'))
                    .orderBy(F.col('sort').asc())
                        )
    else:
        df_t_col = (df_config_table
                   .withColumn('category', F.when(F.col('category').isNull(), F.lit('')).otherwise(F.col('category')))
                   .groupBy('category','column_name','xpath_id').agg(F.min('sort').alias('sort'))
                   .orderBy(F.col('sort').asc())
                        )
            
    #debug   
    display(df_t_col)          
    df_t_col = (df_t_col.alias('r').join(df_config_table.alias('c'),  [F.col('r.xpath_id')==F.col('c.xpath_id')], 'inner')
               .drop('c.xpath_id,c.category,c.column_name,c.sort')
               .orderBy(F.col('r.sort').asc())
               )
    #debug
#     display(df_t_col)

    #generate xpath script , result_type == 1, concat row.  result_type == 2 concat column
    df_t_col_list = df_t_col.collect()

    #init
    with_file_date_col = 'file_date,' if with_file_date==True else ''
    dfs = [] # for type 1
    
    isfristcol = True # for type 2
    w = Window.orderBy(monotonically_increasing_id()) # for type 2

    for r in df_t_col_list:
        xpath = r.xpath 
        xpath_list = xpath.split('/')
        if r.master_detail == 'D':
            if r.xpath == r.raw_group_root:
                xpath_full_text = '/'.join(xpath_list[:-1])+'/'+xpath_list[-1].replace('@','') +'/text()' 
                sql_part_2 = f"xpath(file_content,'{xpath_full_text}')"
            else:
                xpath_full = '/'.join(xpath_list[:-1])+'/@'+xpath_list[-1].replace('@','')  
                sql_part_2 = f"xpath(file_content,'{xpath_full}')"  
        elif (r.master_detail == 'M') :

            xpath_full = '/'.join(xpath_list[:-1])+'/@'+xpath_list[-1].replace('@','') 
            sql_part_2 = f"xpath(file_content,'{xpath_full}')"
        #debug 
        print(sql_part_2)
        
        if result_type == 1:
            sqlscript = f"""select file_id,{with_file_date_col} {sql_part_2} as column_value from TBL_DATA_RAW"""
            _df = ( spark.sql(sqlscript)
                 .withColumn('category',F.lit(r.category))
                 .withColumn('column_value', F.explode('column_value'))
                 .withColumn('column_name',F.lit(r.column_name))
                
                  )
            dfs.append(_df)     
        elif result_type == 2:
            column_name = r.column_name.replace('.','_')
            if isfristcol == True:
                sqlscript = f"""select file_id,{with_file_date_col}{sql_part_2} as {column_name} from TBL_DATA_RAW"""
                parse_result =( spark.sql(sqlscript)
                     .withColumn(column_name, F.explode(column_name))
                      .withColumn("columnindex", row_number().over(w))
                      )
                isfristcol = False
            else:
                sqlscript = f"""select file_id, {sql_part_2} as {column_name} from TBL_DATA_RAW"""
                _df =( spark.sql(sqlscript)
                     .select(F.explode(column_name).alias(column_name))
                     .withColumn("columnindex", row_number().over(w))
                     )
                parse_result = parse_result.join(_df, parse_result.columnindex == _df.columnindex, 'inner').drop(_df.columnindex)

    #output result df type1 unionall / type2 join all column
    
        
    if with_file_date==True:
        collist = ['file_id','file_date','category','column_name','column_value']
    else:
         collist = ['file_id','category','column_name','column_value']
    
    if result_type == 1:
        parse_result = reduce(DataFrame.unionAll, dfs).select([col for col in collist])
    elif result_type == 2:
        parse_result = parse_result.drop(parse_result.columnindex)

    return parse_result


**Reference Code**

In [0]:
# #type 1 testing
# from functools import reduce
# from pyspark.sql import DataFrame

# category = 'Client_Scan_Mode'
# column_name = '32_bit'
# sqlscript1 = """select file_id, xpath(file_content,'/Data/Products/Product/EndPoint/CPU_Type/@CPU_32bit') as column_value from TBL_DATA_RAW"""
# sqlscript2 = """select file_id, xpath(file_content,'/Data/Products/Product/EndPoint/CPU_Type/@CPU_64bit') as column_value from TBL_DATA_RAW"""

# dfs = []
# parse_result_1 =( spark.sql(sqlscript1)
#              .select('file_id', F.explode('column_value').alias('column_value'))
#              .withColumn('category',F.lit(category))
#              .withColumn('column_name',F.lit(column_name))
#               )
# parse_result_2 =( spark.sql(sqlscript2)
#              .select('file_id', F.explode('column_value').alias('column_value'))
#              .withColumn('category',F.lit(category))
#              .withColumn('column_name',F.lit('64_bit'))
#               )
# dfs.append(parse_result_1)
# dfs.append(parse_result_2)

# df = reduce(DataFrame.unionAll, dfs)
 
# # parse_result = unionAll(parse_result_1,parse_result_2)
# display(df)

In [0]:
# #type2 testing

# from pyspark.sql.window import Window
# from pyspark.sql.functions import monotonically_increasing_id, row_number
# w = Window.orderBy(monotonically_increasing_id())

# sqlscript = """select file_id, xpath(file_content,'/Data/Products/Product/Server/PLS/Item/@Version') as Plugin_Version,
# xpath(file_content,'/Data/Products/Product/Server/PLS/Item/@Name') as Plugin_Name, 
# xpath(file_content,'/Data/Products/Product/Server/PLS/Item/@Status') as Plugin_Status
# from TBL_DATA_RAW"""

# parse_result_1 =( spark.sql(sqlscript)
#              .select('file_id',F.explode('Plugin_Version').alias('Plugin_Version'))
#               .withColumn("columnindex", row_number().over(w))
#               )

# parse_result_2 =( spark.sql(sqlscript)
#                .select( F.explode('Plugin_Name').alias('Plugin_Name'))
#                  .withColumn("columnindex", row_number().over(w))
#               )
# parse_result_3 =( spark.sql(sqlscript)
#              .select(F.explode('Plugin_Status').alias('Plugin_Status'))
#                   .withColumn("columnindex", row_number().over(w))
#               )
 

# parse_result_1 = parse_result_1.join(parse_result_2, parse_result_2.columnindex == parse_result_1.columnindex, 'inner').drop(parse_result_2.columnindex)
# parse_result_1 = parse_result_1.join(parse_result_3, parse_result_3.columnindex == parse_result_1.columnindex, 'inner').drop(parse_result_3.columnindex)
# parse_result_1 = parse_result_1.drop(parse_result_1.columnindex)
# display(parse_result_1)